<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-assets/arize-logo-white.jpg" width="200"/>
        <br>
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-assets/phoenix/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://arize.com/docs/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-2w57bhem8-hq24MB6u7yE_ZF_ilOYSBw#/shared-invite/email">Community</a>
    </p>
</center>
<h1 align="center"> Judge Prompt Comparison: Simple/Complex × Reasoning/Non-Reasoning Models </h1>


This notebook uses **Arize Phoenix `llm_classify`** to evaluate tool-calling predictions on the **Berkeley Function Calling Leaderboard (BFCL)** dataset with:
- a **simple** binary prompt (`Yes`/`No`), and
- a **complex** multi-class prompt (`correct` / `partially_correct` / `incorrect`).

We keep it minimal and focused on classification-style LLM-as-a-judge using a non-reasoning model and a reasoning model for the judge. 


## Install & Imports

In [ ]:
%pip -q install --upgrade pandas datasets arize-phoenix openai tiktoken nest_asyncio

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import getpass
import json
import os
import random
import re
import urllib.request
from pathlib import Path

import nest_asyncio
import pandas as pd

from phoenix.evals import llm_classify
from phoenix.evals.models import OpenAIModel

nest_asyncio.apply()

## Configure Judge Models 
 We will be using gpt-4o-mini as our nonreasoning & o3 for the Reasoning Model. Make sure to set your OPENAI_API_KEY. 

In [ ]:
if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass.getpass("🔑 Enter your OpenAI API key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

non_reasoning_model = OpenAIModel(
    model="gpt-4o-mini",
    temperature=0,
)
reasoning_model = OpenAIModel(
    model="o3",
    temperature=0,
)

## Load BFCL (V3 Exec Splits)

>      
> The Berkeley function calling leaderboard is a live leaderboard to evaluate the ability of different LLMs to call functions (also ?referred to as tools). We built this dataset from our learnings to be representative of most users' function calling use-cases, for example, in agents, as a part of enterprise workflows, etc. To this end, our evaluation dataset spans diverse categories, and across multiple languages.
> 

The `exec_simple` dataset is where the 'single function evaluation contains the simplest but most commonly seen format, where the user supplies a single JSON function document, with one and only one function call being invoked.'

The `exec_multiple` dataset is where the 'multiple function category contains a user question that only invokes one function call out of 2 to 4 JSON function documentations. The model needs to be capable of selecting the best function to invoke according to user-provided context.'

More information about these datasets can be found here: https://huggingface.co/datasets/gorilla-llm/Berkeley-Function-Calling-Leaderboard 

In [22]:
BFCL_FILES = {
    "exec_simple": "https://huggingface.co/datasets/gorilla-llm/Berkeley-Function-Calling-Leaderboard/resolve/main/BFCL_v3_exec_simple.json",
    "exec_multiple": "https://huggingface.co/datasets/gorilla-llm/Berkeley-Function-Calling-Leaderboard/resolve/main/BFCL_v3_exec_multiple.json",
}


def fetch_json(url, out_path):
    out = Path(out_path)
    if not out.exists():
        print(f"Downloading {url} -> {out}")
        urllib.request.urlretrieve(url, out)
    text = Path(out).read_text()
    try:
        return json.loads(text)
    except Exception:
        rows = []
        for line in text.splitlines():
            line = line.strip()
            if not line:
                continue
            try:
                rows.append(json.loads(line))
            except Exception:
                pass
        return rows


df_simple = pd.DataFrame(fetch_json(BFCL_FILES["exec_simple"], "BFCL_v3_exec_simple.json"))
df_multi = pd.DataFrame(fetch_json(BFCL_FILES["exec_multiple"], "BFCL_v3_exec_multiple.json"))
print("Simple shape:", df_simple.shape, "Multiple shape:", df_multi.shape)
df = pd.concat([df_simple, df_multi], ignore_index=True)
print("Combined shape:", df.shape)
df.head(2)

Simple shape: (100, 5) Multiple shape: (50, 5)
Combined shape: (150, 5)


,id,question,function,execution_result_type,ground_truth
0,exec_simple_0,"[[{'role': 'user', 'content': 'I've been playi...","[{'name': 'calc_binomial_probability', 'descri...",[exact_match],"[calc_binomial_probability(n=20, k=5, p=0.6)]"
1,exec_simple_1,"[[{'role': 'user', 'content': 'During last nig...","[{'name': 'calc_binomial_probability', 'descri...",[exact_match],"[calc_binomial_probability(n=30, k=15, p=0.5)]"


## Prepare & Format DataFrames
Pull out different parts of the data like, instruction, functions, ground truth, & predictions. 

In [23]:
def extract_gt_call(row):
    gt = row.get("ground_truth")
    if isinstance(gt, list) and gt:
        return gt[0]
    if isinstance(gt, str):
        return gt
    return ""


def extract_functions(row):
    fns = row.get("function", [])
    if isinstance(fns, dict):
        fns = [fns]
    return fns


def extract_instruction(row):
    q = row.get("question", [])
    last_user = ""
    for msg_list in q:
        for m in msg_list:
            if m.get("role") == "user":
                last_user = m.get("content", last_user)
    return last_user


work = []
for _, r in df.iterrows():
    rr = r.to_dict()
    work.append(
        {
            "id": rr.get("id", ""),
            "instruction": extract_instruction(rr),
            "functions_json": json.dumps(
                [
                    {
                        "name": f.get("name"),
                        "parameters": f.get("parameters"),
                        "description": f.get("description", ""),
                    }
                    for f in extract_functions(rr)
                ],
                ensure_ascii=False,
            ),
            "ground_truth": extract_gt_call(rr),
        }
    )
all_data = pd.DataFrame(work)
all_data.head(1)

,id,instruction,functions_json,ground_truth
0,exec_simple_0,I've been playing a game where rolling a six i...,"[{""name"": ""calc_binomial_probability"", ""parame...","calc_binomial_probability(n=20, k=5, p=0.6)"


## Modify Benchmark Dataset

The BFCL Dataset does not have any `negative` examples, i.e. only `question`, `available_tools`, and `ground_truth` are present. In order to accurately benchmark our LLM-as-a-Judge, this code implements a data corruption strategy to generate synthetic evaluation datasets for testing LLM-as-a-Judge systems. It's designed to create realistic "negative examples" (incorrect tool calls) from existing ground truth data, enabling comprehensive evaluation of classification models.

In [24]:
def corrupt_call(s: str) -> str:
    if not s or "(" not in s:
        return s
    tool, args = s.split("(", 1)
    tool = tool.strip()
    args = args.rstrip(")")
    if random.random() < 0.5:
        tool = tool + "_alt"
    else:
        args = re.sub(r"(\d+(?:\.\d+)?)", lambda m: str(float(m.group()) * 1.1), args, count=1)
    return f"{tool} ({args})"


predict_tool_call = [
    gt if random.random() < 0.7 else corrupt_call(gt) for gt in all_data["ground_truth"]
]
data = all_data.copy()
data["predicted_tool_call"] = predict_tool_call

We will be using a small subset of our data for testing purposes. Here we are generating our testing dataset

In [37]:
small_data = data.sample(n=30, random_state=24).reset_index(drop=True)
small_data.head()

,id,instruction,functions_json,ground_truth,predicted_tool_call
0,exec_multiple_7,"As a data analyst, I've been tracking the dail...","[{""name"": ""get_time_zone_by_coord"", ""parameter...","calculate_mean(numbers=[22, 24, 26, 28, 30, 32...","calculate_mean(numbers=[22, 24, 26, 28, 30, 32..."
1,exec_multiple_14,I'm working on a community art project and pla...,"[{""name"": ""calculate_electrostatic_potential_e...",geometry_area_circle(radius=15),geometry_area_circle(radius=15)
2,exec_simple_57,I'm tracking a storm system for my weather rep...,"[{""name"": ""get_time_zone_by_coord"", ""parameter...","get_time_zone_by_coord(long='-80.75', lat='35....","get_time_zone_by_coord(long='-80.75', lat='35...."
3,exec_simple_59,I'm working on a study about climate change in...,"[{""name"": ""get_weather_data"", ""parameters"": {""...","get_weather_data(coordinates=[25.00, 13.00])","get_weather_data(coordinates=[25.00, 13.00])"
4,exec_multiple_5,"During a simulation of a high-speed pursuit, I...","[{""name"": ""calculate_cosine_similarity"", ""para...","calculate_final_velocity(initial_velocity=0, a...",calculate_final_velocity (initial_velocity=0.0...


## Define your LLM-as-a-Judge Templates & Rails

In [42]:
SIMPLE_TEMPLATE = """You are grading a tool-calling attempt.

Given:
USER INSTRUCTION:
{instruction}

AVAILABLE FUNCTIONS (JSON Schemas):
{functions_json}

MODEL TOOL CALL (string):
{predicted_tool_call}

GROUND TRUTH TOOL CALL (string):
{ground_truth}

Question: Did the model invoke the correct tool(s) AND use the correct parameter names and values?
Answer strictly with one word, Yes or No, & an explanation for your answer.

Example response:
LABEL: "Yes" or "No"
EXPLANATION: An explanation of your reasoning for why the label is "Yes" or "No"
"""

SIMPLE_RAILS = ["Yes", "No"]

In [ ]:
COMPLEX_TEMPLATE = """You are grading a tool-calling attempt.
Return ONLY one of the following labels:
- correct
- partially_correct
- incorrect

Use these rules:
- Consider types and trivial formatting (e.g., '5' vs 5, whitespace) as equivalent.
- Consider equivalent units only if explicitly clear from context.
- The attempt is "correct" only if the tool and all required parameters match the ground truth.
- It's "partially_correct" if the tool is correct but parameters have minor issues.
- It's "incorrect" otherwise.

Context:
USER INSTRUCTION:
{instruction}

AVAILABLE FUNCTIONS (JSON Schemas):
{functions_json}

MODEL TOOL CALL (string):
{predicted_tool_call}

GROUND TRUTH TOOL CALL (string):
{ground_truth}

Question: Use the rules above to determine if the model's tool call is correct, partially correct, or incorrect.
Answer strictly with one label & an explanation for your answer.

Example response:
LABEL: "correct" or "partially_correct" or "incorrect"
EXPLANATION: An explanation of your reasoning for why the label is "correct" or "partially_correct" or "incorrect"
"""

COMPLEX_RAILS = ["correct", "partially_correct", "incorrect"]

## Run our Simple Evaluation on both Judge Models

In [44]:
simple_df = small_data.copy()

non_reasoning_simple_results = llm_classify(
    data=simple_df.assign(template=SIMPLE_TEMPLATE),
    model=non_reasoning_model,
    template="{template}",
    rails=SIMPLE_RAILS,
    provide_explanation=True,
    include_prompt=False,
    include_response=True,
    run_sync=True,
)

non_reasoning_simple_results.head(3)

llm_classify |          | 0/30 (0.0%) | ⏳ 00:00<? | ?it/s

,label,explanation,response,exceptions,execution_status,execution_seconds,prompt_tokens,completion_tokens,total_tokens
0,no,The model did not invoke the correct tool or u...,"{""explanation"":""The model did not invoke the c...",[],COMPLETED,1.602984,194,50,244
1,no,The model did not invoke the correct tool or u...,"{""explanation"":""The model did not invoke the c...",[],COMPLETED,1.309475,194,50,244
2,no,The model invoked the correct tool but used in...,"{""explanation"":""The model invoked the correct ...",[],COMPLETED,1.034774,194,22,216


In [52]:
simple_df = small_data.copy()

reasoning_simple_results = llm_classify(
    data=simple_df.assign(template=SIMPLE_TEMPLATE),
    model=reasoning_model,
    template="{template}",
    rails=SIMPLE_RAILS,
    provide_explanation=True,
    include_prompt=False,
    include_response=True,
    run_sync=True,
)

reasoning_simple_results.head(3)

llm_classify |          | 0/30 (0.0%) | ⏳ 00:00<? | ?it/s

,label,explanation,response,exceptions,execution_status,execution_seconds,prompt_tokens,completion_tokens,total_tokens
0,no,"The necessary details (user instruction, avail...","{""response"":""No"",""explanation"":""The necessary ...",[],COMPLETED,6.347308,188,326,514
1,no,The model’s tool invocation does not exactly m...,"{""response"":""No"",""explanation"":""The model’s to...",[],COMPLETED,19.295503,188,1088,1276
2,no,The prompt did not provide the user instructio...,"{""response"":""No"",""explanation"":""The prompt did...",[],COMPLETED,3.337270,188,194,382


## Run our Complex Evaluation on both Judge Models

In [50]:
complex_df = small_data.copy()

non_reasoning_complex_results = llm_classify(
    data=complex_df.assign(template=COMPLEX_TEMPLATE),
    model=non_reasoning_model,
    template="{template}",
    rails=COMPLEX_RAILS,
    provide_explanation=True,
    include_prompt=False,
    include_response=True,
    run_sync=True,
)
non_reasoning_complex_results.head(3)

llm_classify |          | 0/30 (0.0%) | ⏳ 00:00<? | ?it/s

,label,explanation,response,exceptions,execution_status,execution_seconds,prompt_tokens,completion_tokens,total_tokens
0,incorrect,The model's tool call does not match the groun...,"{""explanation"":""The model's tool call does not...",[],COMPLETED,1.319789,311,38,349
1,incorrect,The model's tool call does not match the groun...,"{""explanation"":""The model's tool call does not...",[],COMPLETED,1.662194,311,38,349
2,incorrect,The model's tool call does not match the groun...,"{""explanation"":""The model's tool call does not...",[],COMPLETED,1.137608,311,38,349


In [48]:
complex_df = small_data.copy()

reasoning_complex_results = llm_classify(
    data=complex_df.assign(template=COMPLEX_TEMPLATE),
    model=reasoning_model,
    template="{template}",
    rails=COMPLEX_RAILS,
    provide_explanation=True,
    include_prompt=False,
    include_response=True,
    run_sync=True,
)

reasoning_complex_results.head(3)

llm_classify |          | 0/30 (0.0%) | ⏳ 00:00<? | ?it/s

,label,explanation,response,exceptions,execution_status,execution_seconds,prompt_tokens,completion_tokens,total_tokens
0,incorrect,The necessary information to make an accurate ...,"{""response"":""incorrect"",""explanation"":""The nec...",[],COMPLETED,17.276113,305,1263,1568
1,incorrect,Unable to evaluate: the required ground-truth ...,"{""response"":""incorrect"",""explanation"":""Unable ...",[],COMPLETED,11.018556,305,632,937
2,incorrect,"The necessary details of the user instruction,...","{""response"":""incorrect"",""explanation"":""The nec...",[],COMPLETED,5.915661,305,269,574


## View Results

We will compare the number of times the models disagree on their evaluation labels as well as how many tokens they used to complete their evaluations.

In [55]:
print("For Simple Eval: ")
print("-----------------------------------------------------------")

simple_different_labels = (
    non_reasoning_simple_results["label"] != reasoning_simple_results["label"]
).sum()
if simple_different_labels == 0:
    print("Reasoning and non-reasoning models agree on all samples")
else:
    print(f"Reasoning and non-reasoning models disagree on {simple_different_labels} samples")
NR_simple_tokens = non_reasoning_simple_results["total_tokens"].sum()
R_simple_tokens = reasoning_simple_results["total_tokens"].sum()

print(f"Non-reasoning model used {NR_simple_tokens} tokens")
print(f"Reasoning model used {R_simple_tokens} tokens")
print(
    f"Reasoning model is {R_simple_tokens / NR_simple_tokens} times more expensive than the non-reasoning model"
)

For Simple Eval: 
-----------------------------------------------------------
Reasoning and non-reasoning models agree on all samples
Non-reasoning model used 7238 tokens
Reasoning model used 21644 tokens
Reasoning model is 2.990328820116054 times more expensive than the non-reasoning model


In [56]:
print("For Complex Eval: ")
print("-----------------------------------------------------------")

complex_different_labels = (
    non_reasoning_complex_results["label"] != reasoning_complex_results["label"]
).sum()
if complex_different_labels == 0:
    print("Reasoning and non-reasoning models agree on all samples")
else:
    print(f"Reasoning and non-reasoning models disagree on {complex_different_labels} samples")
NR_complex_tokens = non_reasoning_complex_results["total_tokens"].sum()
R_complex_tokens = reasoning_complex_results["total_tokens"].sum()

print(f"Non-reasoning model used {NR_complex_tokens} tokens")
print(f"Reasoning model used {R_complex_tokens} tokens")
print(
    f"Reasoning model is {R_complex_tokens / NR_complex_tokens} times more expensive than the non-reasoning model"
)

For Complex Eval: 
-----------------------------------------------------------
Reasoning and non-reasoning models disagree on 1 samples
Non-reasoning model used 10479 tokens
Reasoning model used 25355 tokens
Reasoning model is 2.4196011069758563 times more expensive than the non-reasoning model


### References
- Phoenix Evals Overview: https://arize.com/docs/phoenix/evaluation/llm-evals
- Using `llm_classify` (Docs): https://arize.com/docs/phoenix/evaluation/how-to-evals/bring-your-own-evaluator
- BFCL dataset: https://huggingface.co/datasets/gorilla-llm/Berkeley-Function-Calling-Leaderboard
